In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
follows = pd.read_csv("follows.csv")
follows

,user_id,follows,date
0,1,2,1993-09-01
1,2,1,1989-01-01
2,3,1,1993-07-01
3,2,3,1994-10-10
4,3,2,1995-03-01
5,4,2,1988-08-08
6,4,1,1988-08-08
7,1,4,1994-04-02
8,1,5,2000-01-01
9,5,1,2000-01-02


In [3]:
users = pd.read_csv("users.csv")
users["full_name"] = users["first_name"] + " " + users["last_name"]
users

,user_id,first_name,last_name,house,full_name
0,1,Harry,Potter,Gryffindor,Harry Potter
1,2,Ron,Wesley,Gryffindor,Ron Wesley
2,3,Hermonie,Granger,Gryffindor,Hermonie Granger
3,4,Ginny,Weasley,Gryffindor,Ginny Weasley
4,5,Draco,Malfoy,Slytherin,Draco Malfoy
5,6,Tom,Riddle,Slytherin,Tom Riddle
6,7,Luna,Lovegood,Ravenclaw,Luna Lovegood
7,8,Cho,Chang,Ravenclaw,Cho Chang
8,9,Cedric,Diggory,Hufflepuff,Cedric Diggory


In [4]:
my_dfs = {"follows": follows, "users": users}

with sqlite3.connect("data.sqlite") as my_db:
    for table_name, df in my_dfs.items():
        df.to_sql(table_name, my_db, if_exists = "replace")

conn = sqlite3.connect("data.sqlite")
db = conn.cursor()

# How many users are there in each house?

In [5]:
query = """
    SELECT house, COUNT(house)
    FROM users
    GROUP BY house
"""

db.execute(query)
users_per_house = db.fetchall()
users_per_house

[('Gryffindor', 4), ('Hufflepuff', 1), ('Ravenclaw', 2), ('Slytherin', 2)]

# List all following links that were created before September 1st, 1993.

In [7]:
query = """
    SELECT user_id, follows, date
    FROM follows
    WHERE date < '1993-09-01'
"""

db.execute(query)
links_created_before_1993_09_01 = db.fetchall()
links_created_before_1993_09_01

[(2, 1, '1989-01-01'),
 (3, 1, '1993-07-01'),
 (4, 2, '1988-08-08'),
 (4, 1, '1988-08-08'),
 (5, 6, '1986-01-10'),
 (7, 1, '1990-02-02')]

# List all the following links established before September 1st 1993, but this time use the users' first names.

In [8]:
query = """
    SELECT uid.first_name, uf.first_name, follows.date
    FROM follows
    LEFT JOIN users uid ON uid.user_id = follows.user_id
    LEFT JOIN users uf ON uf.user_id = follows.follows
    WHERE follows.date < '1993-09-01'
"""

db.execute(query)
links_created_before_1993_09_01_first_names = db.fetchall()
links_created_before_1993_09_01_first_names

[('Ron', 'Harry', '1989-01-01'),
 ('Hermonie', 'Harry', '1993-07-01'),
 ('Ginny', 'Ron', '1988-08-08'),
 ('Ginny', 'Harry', '1988-08-08'),
 ('Draco', 'Tom', '1986-01-10'),
 ('Luna', 'Harry', '1990-02-02')]

# Give a count of how many people followed each user as of 1999-12-31. Give the result in term of "users' full name, number of followers."

In [9]:
query = """
    SELECT users.full_name, COUNT(follows.follows)
    FROM follows
    JOIN users on users.user_id = follows.user_id
    WHERE follows.date <= '1999-12-31'
    GROUP BY users.user_id
"""

db.execute(query)
number_of_followers_1999_12_31 = db.fetchall()
number_of_followers_1999_12_31

[('Harry Potter', 4),
 ('Ron Wesley', 2),
 ('Hermonie Granger', 3),
 ('Ginny Weasley', 3),
 ('Draco Malfoy', 1),
 ('Luna Lovegood', 2),
 ('Cho Chang', 2),
 ('Cedric Diggory', 1)]

# List the number of users each user follows.

In [10]:
query = """
    SELECT users.full_name, COUNT(follows.user_id)
    FROM follows
    JOIN users on users.user_id = follows.follows
    GROUP BY users.user_id
"""

db.execute(query)
number_followed = db.fetchall()
number_followed

[('Harry Potter', 6),
 ('Ron Wesley', 3),
 ('Hermonie Granger', 1),
 ('Ginny Weasley', 1),
 ('Draco Malfoy', 1),
 ('Tom Riddle', 1),
 ('Luna Lovegood', 1),
 ('Cho Chang', 3),
 ('Cedric Diggory', 3)]

# List all rows from follows where someone from one house follows someone from a different house. Please give the result with user names.

In [11]:
query = """
    SELECT uid.full_name, uid.house, uf.full_name, uf.house
    FROM follows
    LEFT JOIN users uid on uid.user_id = follows.user_id
    LEFT JOIN users uf on uf.user_id = follows.follows
    WHERE uid.house != uf.house
"""

db.execute(query)
followed_different_house = db.fetchall()
followed_different_house

[('Harry Potter', 'Gryffindor', 'Draco Malfoy', 'Slytherin'),
 ('Draco Malfoy', 'Slytherin', 'Harry Potter', 'Gryffindor'),
 ('Luna Lovegood', 'Ravenclaw', 'Harry Potter', 'Gryffindor'),
 ('Harry Potter', 'Gryffindor', 'Luna Lovegood', 'Ravenclaw'),
 ('Harry Potter', 'Gryffindor', 'Cho Chang', 'Ravenclaw'),
 ('Cho Chang', 'Ravenclaw', 'Harry Potter', 'Gryffindor'),
 ('Cho Chang', 'Ravenclaw', 'Cedric Diggory', 'Hufflepuff'),
 ('Cedric Diggory', 'Hufflepuff', 'Cho Chang', 'Ravenclaw'),
 ('Hermonie Granger', 'Gryffindor', 'Cedric Diggory', 'Hufflepuff'),
 ('Ginny Weasley', 'Gryffindor', 'Cedric Diggory', 'Hufflepuff')]

# List all unrequited followings (i.e. where A follows B but B does not follow A).

In [12]:
query = """
    SELECT uid.full_name, uf.full_name
    FROM follows
    LEFT JOIN users uid on uid.user_id = follows.user_id
    LEFT JOIN users uf on uf.user_id = follows.follows
"""

db.execute(query)
all_followings = db.fetchall()
all_followings

unrequited_followings = []
for pair in all_followings:
    flipped_pair = (pair[1], pair[0])
    if flipped_pair not in all_followings:
        unrequited_followings.append(pair)

unrequited_followings

[('Hermonie Granger', 'Harry Potter'),
 ('Ginny Weasley', 'Ron Wesley'),
 ('Draco Malfoy', 'Tom Riddle'),
 ('Luna Lovegood', 'Cho Chang'),
 ('Hermonie Granger', 'Cedric Diggory'),
 ('Ginny Weasley', 'Cedric Diggory')]